# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port macquarie,-31.4333,152.9167,74.89,87,97,8.39,AU,1706545844
1,1,uturoa,-16.7333,-151.4333,82.78,76,92,3.06,PF,1706545844
2,2,narasannapeta,18.4167,84.0500,69.71,71,41,1.61,IN,1706545845
3,3,kununurra,-15.7667,128.7333,82.38,83,79,0.00,AU,1706545845
4,4,new norfolk,-42.7826,147.0587,51.71,97,9,2.98,AU,1706545845


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [40]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points('Lng'
                                               ,'Lat'
                                               ,geo = True
                                               ,tiles = 'OSM'
                                               ,size = city_data_df['Humidity']
                                               ,frame_width = 700
                                               ,frame_height = 500
                                               ,color = 'City'
                                               ,cmap='colorwheel'
                                               ,hover_cols = ['City', 'Humidity'])


# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = pd.DataFrame(city_data_df[(city_data_df['Max Temp'] < 75) 
                             & (city_data_df['Humidity'] < 65)
                             & (city_data_df['Max Temp'] > 60)
                             & (city_data_df['Cloudiness'] > 40)
                            ])

# Drop any rows with null values
ideal_weather.dropna(inplace = True)

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,puerto natales,-51.7236,-72.4875,64.85,48,75,20.71,CL,1706545849
85,85,dalbandin,28.8947,64.4101,65.35,48,100,5.21,PK,1706545858
89,89,ad dabbah,18.0500,30.9500,67.64,18,45,10.83,SD,1706545859
96,96,lebu,-37.6167,-73.6500,65.14,60,61,12.03,CL,1706545860
120,120,kasba tadla,32.5977,-6.2684,73.51,24,98,5.84,MA,1706545694


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
28,puerto natales,CL,-51.7236,-72.4875,48,
85,dalbandin,PK,28.8947,64.4101,48,
89,ad dabbah,SD,18.0500,30.9500,18,
96,lebu,CL,-37.6167,-73.6500,60,
120,kasba tadla,MA,32.5977,-6.2684,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel'
    ,'limit': 1
    ,'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_response = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto natales - nearest hotel: Dorotea Patagonia Hostel
dalbandin - nearest hotel: SF Al-dawood Restaurant
ad dabbah - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
kasba tadla - nearest hotel: No hotel found
itoman - nearest hotel: No hotel found
llanes - nearest hotel: Los Molinos
gondia - nearest hotel: The Grand Sita
puerto aysen - nearest hotel: Hostal Dendal
ancud - nearest hotel: Cabañas
chambas - nearest hotel: Hotel Colón
arteaga - nearest hotel: No hotel found
rodolfo sanchez taboada - nearest hotel: Estero Beach Resort
katsuren-haebaru - nearest hotel: No hotel found
bahia de kino - nearest hotel: Hotel Hacienda
tumayr - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
28,puerto natales,CL,-51.7236,-72.4875,48,Dorotea Patagonia Hostel
85,dalbandin,PK,28.8947,64.4101,48,SF Al-dawood Restaurant
89,ad dabbah,SD,18.0500,30.9500,18,No hotel found
96,lebu,CL,-37.6167,-73.6500,60,No hotel found
120,kasba tadla,MA,32.5977,-6.2684,24,No hotel found
135,itoman,JP,26.1247,127.6694,63,No hotel found
139,llanes,ES,43.4198,-4.7549,63,Los Molinos
165,gondia,IN,21.4500,80.2000,33,The Grand Sita
199,puerto aysen,CL,-45.4000,-72.7000,62,Hostal Dendal
256,ancud,CL,-41.8697,-73.8203,56,Cabañas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points('Lng'
                                        ,'Lat'
                                        ,geo = True
                                        ,tiles = 'OSM'
                                        ,size = hotel_df['Humidity']
                                        ,frame_width = 700
                                        ,frame_height = 500
                                        ,color = 'City'
                                        ,cmap='colorwheel'
                                        ,hover_cols = ['City', 'Country', 'Humidity', 'Hotel Name'])

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,_size,Country,Humidity,Hotel Name)